# W11 Lab Assignment

Maps

## Basemap

`Basemap` (official documention [here](http://matplotlib.org/basemap/index.html), unofficial documentation [here](http://basemaptutorial.readthedocs.io/en/latest/index.html) ) is a Python package for visualization with maps. To install it, open your terminal and run:

`conda install basemap`

In [ ]:
from mpl_toolkits.basemap import Basemap
%matplotlib inline

Now we can draw our first map:

In [ ]:
map = Basemap()
map.drawcoastlines()

The default projection is the [Equirectangular projection](https://en.wikipedia.org/wiki/Equirectangular_projection). We can change to another projection:

In [ ]:
map = Basemap(projection='ortho', lat_0=0, lon_0=0)
map.drawcoastlines()

`lat_0` and `lon_0` sets the center of the map. Changing it allows us to see the other parts of the earth.

In [ ]:
map = Basemap(projection='ortho', lat_0=0, lon_0=90)
map.drawcoastlines()

Fill the continents and oceans with some colors:

In [ ]:
map = Basemap(projection='ortho', lat_0=0, lon_0=0)
map.drawmapboundary(fill_color='aqua')
map.fillcontinents(color='coral',lake_color='aqua')
map.drawcoastlines()

Add country lines:

In [ ]:
map = Basemap(projection='ortho', lat_0=0, lon_0=0)
map.drawmapboundary(fill_color='aqua')
map.fillcontinents(color='coral',lake_color='aqua')
map.drawcoastlines()
map.drawcountries()

The map is essentially a matplotlib object, so we can use matplotlib's methods to access it, for example, draw some points on the map.

In [ ]:
map = Basemap(projection='ortho', lat_0=0, lon_0=-30)
map.drawmapboundary(fill_color='aqua')
map.fillcontinents(color='coral',lake_color='aqua')
map.drawcoastlines()
x, y = map(-10, 0) #x=longitude, y=latitude
map.plot(x, y, marker='D',color='m')

Try with this: put a dot on your favourite city and put its name next to it. You can use the [text](http://basemaptutorial.readthedocs.io/en/latest/plotting_data.html#text) function. (It may be necessary to rotate your map).

In [ ]:
#TODO: draw a point for a city on the map.

It is also possible to only draw a region within a given "box" by passing the coordinates of the lower-left corner and upper-right corner.

In [ ]:
map = Basemap(llcrnrlon=-10.5,llcrnrlat=35,urcrnrlon=4.,urcrnrlat=44.,
             resolution='i', projection='tmerc', lat_0 = 39.5, lon_0 = -3.25)

map.drawmapboundary(fill_color='aqua')
map.fillcontinents(color='coral',lake_color='aqua')
map.drawcoastlines()

In [ ]:
#TODO: plot the region with the city that you just plotted at the center.